# 基本功能

## 摘要
1. 通过httpx代替requests进行请求
2. 教务系统登录login
3. 查询今天要上的课get_classes_day() 默认今天
4. 查询指定日期的课程get_classes_day("指定日期")，日期格式：2024.01.02
5. 输入学期+周查询周课表get_classes_from_week("2022.1","周次")，不输入默认本周
6. 查询绩点情况get_grade_count()
7. 根据学期查询各课绩点情况get_grade("2023-1")，必填

## 相关代码实现
引入相关库

In [3]:
from pydantic import 

import json
import re
from bs4 import BeautifulSoup
import httpx
import asyncio
import datetime

相关链接url

In [4]:
ecjtu_login = "http://cas.ecjtu.edu.cn/cas/login" # 智慧交大登录页
jwxt_login = "https://jwxt.ecjtu.edu.cn/stuMag/Login_dcpLogin.action" # 教务系统登录页
ecjtu_to_jwxt = "http://cas.ecjtu.edu.cn/cas/login?service=https%3A%2F%2Fjwxt.ecjtu.edu.cn%2FstuMag%2FLogin_dcpLogin.action" #智慧交大登录教务系统
get_classes = "https://jwxt.ecjtu.edu.cn/Schedule/Weekcalendar_getTodayWeekcalendar.action" # 获取课程信息(post)
get_grade = "https://jwxt.ecjtu.edu.cn/scoreQuery/stuScoreQue_getStuScore.action?item=0401" # 获取成绩信息(get)

### 1. 用户登录login
1. 获取加密密码

In [5]:
def get_enc_password(origin_pwd: str) -> str:
    ENC_URL = 'http://cas.ecjtu.edu.cn/cas/loginPasswdEnc'
    enc_response = httpx.post(ENC_URL, data={'pwd': origin_pwd})

    _ = enc_response.content.decode('utf8').replace("'", '"')
    return json.loads(_)['passwordEnc']

password = "your_password"
print(get_enc_password(password))

30052603102467EDE4DA3F7C61F023727039121D4EC611A63A537C2277D34897F693CC5142911946DFDD34206A55049E


2. 登录login

In [6]:
async def login(username: str, password: str, session: httpx.AsyncClient) -> httpx.AsyncClient:
    """登录教务系统"""

    # 获取加密密码
    enc_password = get_enc_password(password)
    # 打包登录body
    login_data = {
        "username": username,
        "password": enc_password,
        "lt": "",
        "service": "http://portal.ecjtu.edu.cn/dcp/index.jsp",
    }
    # 获取lt
    headers = {
        # 设置UA
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
        # 设置Host
        "Host": "cas.ecjtu.edu.cn",
    }
    response =await session.get(ecjtu_login, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    login_data["lt"] = soup.find("input", {"name": "lt"})["value"]
    # 封包headers
    headers_append = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Referer": "http://cas.ecjtu.edu.cn/cas/login",
    }
    headers.update(headers_append)
    # 发送登录请求
    response = await session.post(ecjtu_login, data=login_data, headers=headers)
    # 能拿到cookie中的CASTGC则登录成功
    if not response.cookies.get("CASTGC"):
        print("登录失败")
        return None
    # 登录教务系统
    response = await session.get(jwxt_login, headers=headers)
    # 获取教务系统重定向
    response_url =await session.get(ecjtu_to_jwxt)
    # 绑定教务系统cookie
    result =await session.get(response_url.headers['Location'],follow_redirects=True)
    if result.status_code != 200:
        print("教务系统错误")
        return None
    print("登录成功")
    return session

async def main():
    client = httpx.AsyncClient(verify=False)
    session = await login("2022211003000111", "mzdwzyddn0", client)


# if __name__ == "__main__":
    # asyncio.run(main())
await main()    # 在jupyter notebook中使用asyncio.run会报错，所以使用await main()代替



登录成功


3. 查询指定日期课程（get_classes_day）

In [15]:
async def get_classes_day(session: httpx.AsyncClient,date:str = datetime.datetime.now().strftime("%Y-%m-%d"))->list:
    """获取指定课程"""
    data = {
        "date": date
    }
    classes =await session.post(get_classes, data=data)
    if len(classes.json()["weekcalendarpojoList"]) == 0:
        return "今天没有课"
    # 返回今天的课（课程时间，课程名，课程教室，老师名，是否实验)
    class_list = []
    for item in classes.json()["weekcalendarpojoList"]:
        class_info = {
                "课程时间": item["classString"],
                "课程名": item["className"],
                "课程教室": item["classRoom"],
                "老师名": item["teacherName"],
                "是否实验": item["pkType"],
        }
        class_list.append(class_info)
        # class_info = {
        #         "class_time": item["classString"],
        #         "class_name": item["className"],
        #         "class_room": item["classRoom"],
        #         "class_teacher": item["teacherName"],
        #         "is_experiment": item["pkType"],
        # }
    return class_list

client = httpx.AsyncClient(verify=False)
session = await login("2022211003000111", "mzdwzyddn0", client)
print(await get_classes_day(session=session))
print(await get_classes_day(session=session,date="2024-03-07"))
respose = await session.post(get_classes, data={"date": "2024-03-07"})
print(respose.json())





登录成功
[{'课程时间': '5-6', '课程名': '排球Ⅳ(20232-7)', '课程教室': '北区田径场25', '老师名': '梁日尉', '是否实验': '上课'}, {'课程时间': '7-8', '课程名': '生活中的经济学(20232-1)', '课程教室': '31-204', '老师名': '刘志虹', '是否实验': '上课'}]
[{'课程时间': '1-2', '课程名': 'IB级(2022-13)', '课程教室': '31-612', '老师名': '陶国潇', '是否实验': '上课'}, {'课程时间': '3-4', '课程名': '线性代数B(20232-1)', '课程教室': '31-501', '老师名': '詹小秦', '是否实验': '上课'}]
{'date': '2024-03-07', 'dsWeek': 2, 'userflag': 0, 'week': 2, 'weekcalendarpojoList': [{'classString': '1-2', 'courseRequire': '必修课', 'teacherName': '陶国潇', 'weekDay': 4, 'classRoom': '31-612', 'classSpan': '1,2', 'course': '大学英语Ⅳ', 'className': 'IB级(2022-13)', 'weekSpan': '1-16', 'pkType': '上课'}, {'classString': '3-4', 'courseRequire': '必修课', 'teacherName': '詹小秦', 'weekDay': 4, 'classRoom': '31-501', 'classSpan': '3,4', 'course': '线性代数B', 'className': '线性代数B(20232-1)', 'weekSpan': '1-16', 'pkType': '上课'}], 'weekDay': '4'}


4. 输入学期查询周课表get_classes_from_week

In [ ]:
def get_classes_from_week(week: int)